In [ ]:
from dictdiffer import diff
from gql import Client, gql
from gql.transport.requests import RequestsHTTPTransport
from phoenix.server.api.types.pagination import (
    cursor_to_offset,
    offset_to_cursor,
)

new_url = "http://127.0.0.1:6006/graphql"
old_url = "http://127.0.0.1:6008/graphql"

new_project_id = "UHJvamVjdDox"
old_project_id = "UHJvamVjdDow"

new_client = Client(
    transport=RequestsHTTPTransport(url=new_url, timeout=1),
    fetch_schema_from_transport=True,
)
old_client = Client(
    transport=RequestsHTTPTransport(url=old_url, timeout=1),
    fetch_schema_from_transport=True,
)

In [ ]:
spans_query = gql(
    """query SpansQuery($projectId: GlobalID!, $after: String = null, $before: String = null, $filterCondition: String = null, $first: Int = null, $last: Int = null, $sort: SpanSort = null) {
  node(id: $projectId) {
    ... on Project {
      spans(
        after: $after
        before: $before
        filterCondition: $filterCondition
        first: $first
        last: $last
        rootSpansOnly: false
        sort: $sort
      ) {
        edges {
          cursor
        }
        pageInfo {
          hasNextPage
          hasPreviousPage
        }
      }
    }
  }
}"""
)


def compare_responses(new_response, old_response):
    new_cursors = [edge["cursor"] for edge in new_response["node"]["spans"]["edges"]]
    new_ids = [cursor_to_offset(cursor) for cursor in new_cursors]
    old_cursors = [edge["cursor"] for edge in old_response["node"]["spans"]["edges"]]
    old_offsets = [cursor_to_offset(cursor) for cursor in old_cursors]
    new_page_info = new_response["node"]["spans"]["pageInfo"]
    old_page_info = old_response["node"]["spans"]["pageInfo"]
    page_info_diff = list(diff(new_page_info, old_page_info))
    print(f"{new_ids=}")
    print(f"{old_offsets=}")
    print(f"{page_info_diff=}")
    assert new_ids == [offset + 1 for offset in old_offsets], "mismatched spans"
    assert page_info_diff == [], "mismatched page info"

In [ ]:
# new_response = new_client.execute(
#     spans_query,
#     variable_values={"projectId": new_project_id, "first": 10},
# )
# old_response = old_client.execute(
#     spans_query,
#     variable_values={"projectId": old_project_id, "first": 10},
# )
# compare_responses(new_response, old_response)

In [ ]:
new_response = new_client.execute(
    spans_query,
    variable_values={
        "projectId": new_project_id,
        "after": offset_to_cursor(2),
        "first": 10,
    },
)
old_response = old_client.execute(
    spans_query,
    variable_values={
        "projectId": old_project_id,
        "after": offset_to_cursor(1),
        "first": 10,
    },
)
compare_responses(new_response, old_response)

In [ ]:
# there are 765 spans in the llama-index rag fixture
new_response = new_client.execute(
    spans_query,
    variable_values={
        "projectId": new_project_id,
        "after": offset_to_cursor(754),
        "first": 10,
    },
)
old_response = old_client.execute(
    spans_query,
    variable_values={
        "projectId": old_project_id,
        "after": offset_to_cursor(753),
        "first": 10,
    },
)
compare_responses(new_response, old_response)

In [ ]:
# there are 765 spans in the llama-index rag fixture
new_response = new_client.execute(
    spans_query,
    variable_values={
        "projectId": new_project_id,
        "after": offset_to_cursor(755),
        "first": 10,
    },
)
old_response = old_client.execute(
    spans_query,
    variable_values={
        "projectId": old_project_id,
        "after": offset_to_cursor(754),
        "first": 10,
    },
)
compare_responses(new_response, old_response)

In [ ]:
# there are 765 spans in the llama-index rag fixture
new_response = new_client.execute(
    spans_query,
    variable_values={
        "projectId": new_project_id,
        "after": offset_to_cursor(756),
        "first": 10,
    },
)
old_response = old_client.execute(
    spans_query,
    variable_values={
        "projectId": old_project_id,
        "after": offset_to_cursor(755),
        "first": 10,
    },
)
compare_responses(new_response, old_response)

In [ ]:
new_response = new_client.execute(
    spans_query,
    variable_values={
        "projectId": new_project_id,
        "after": offset_to_cursor(2),
        "first": 10,
        "filterCondition": "span_kind == 'LLM'",
    },
)
new_cursors = [edge["cursor"] for edge in new_response["node"]["spans"]["edges"]]
new_ids = [cursor_to_offset(cursor) for cursor in new_cursors]
assert new_ids == [6, 11, 16, 21, 26, 31, 36, 41, 46, 51], new_ids

In [ ]:
new_response = new_client.execute(
    spans_query,
    variable_values={
        "projectId": new_project_id,
        "sort": {"col": "startTime", "dir": "desc"},
        "first": 10,
    },
)
new_cursors = [edge["cursor"] for edge in new_response["node"]["spans"]["edges"]]
new_ids = [cursor_to_offset(cursor) for cursor in new_cursors]
assert new_ids == [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], new_ids

In [ ]:
new_response = new_client.execute(
    spans_query,
    variable_values={
        "projectId": new_project_id,
        "sort": {"col": "startTime", "dir": "asc"},
        "first": 10,
    },
)
new_cursors = [edge["cursor"] for edge in new_response["node"]["spans"]["edges"]]
new_ids = [cursor_to_offset(cursor) for cursor in new_cursors]
assert new_ids == [765, 764, 763, 762, 761, 760, 759, 758, 757, 756], new_ids